## Working with Snapshot camera

Idea: find only the large cluster to reduce detection noise. Then, use the lifespan of customer's cluster to find the most probable vehicle in the snapshot.
Snapshot cam: takes photo every 1:15 second

In [1]:
import sys

sys.path.append("/home/jeans/internship/parking-customer-count")

from countpassenger.Config import conf
from countpassenger import Preprocess
from countpassenger import Cluster
from countpassenger import Utils
from countpassenger import Snapshot
from countpassenger import Dataset

import countpassenger
import pandas as pd
import numpy as np

import os.path as osp
import os

pd.set_option("display.precision", 10)


CURRENT_CAM = "mbk-14-11"
SAVE_DIR = os.path.join(
    conf.RESOURCES_PROCESSED_DIR, f"2024-04-29/count-passengers-{CURRENT_CAM}-vehicle-snapshot.csv"
)

load_param = dict(
    vehicle_path_rel="2024-04-29/mbk-tourist-vehicle-object-20240429.csv",
    cross_path_rel="2024-04-29/mbk-tourist-raw-cross-object-20240429.csv",
    reverse_path_rel="2024-04-29/mbk-tourist-raw-reverse-object-20240429.csv",
)

In [2]:
start = 1714284000
stop = 1714288000

df_vehicle, df_cross, df_reverse = Dataset.load_dataset_from_paths(**load_param)

df_vehicle = Preprocess.df_clean_vehicle(
    df_vehicle=df_vehicle,
)
df_vehicle = Preprocess.filter_camera(df_vehicle, camera_name=CURRENT_CAM + "-vehicle-snapshot")
df_vehicle = Preprocess.sort_df(df_vehicle, conf.VEHICLE_INTEREST_SNAPSHOT)
df_vehicle = Preprocess.snapshot_drop_parked(df_vehicle)

# print(df_vehicle[[*conf.VEHICLE_INTEREST_SNAPSHOT]].to_string())

df_cross = Preprocess.df_clean_customer(df_cross)
df_cross = Preprocess.filter_camera(df_cross, camera_name=CURRENT_CAM)
df_cross = Preprocess.sort_df(df_cross, conf.CUSTOMER_INTEREST_SNAPSHOT)

# print(df_cross[[*conf.CUSTOMER_INTEREST_SNAPSHOT, "timestamp_unix"]].to_string())

df_reverse = Preprocess.df_clean_customer(df_reverse)
df_reverse = Preprocess.filter_camera(df_reverse, camera_name=CURRENT_CAM)
df_reverse = Preprocess.sort_df(df_reverse, conf.CUSTOMER_INTEREST_SNAPSHOT)

# print(df_reverse[[*conf.CUSTOMER_INTEREST_SNAPSHOT, "timestamp_unix"]].to_string())

### Cluster only large vehicle 

In [3]:
params = dict(
    min_cluster_size=2,  # have at least 5 people together
    min_samples=2,
    cluster_selection_epsilon=0,
    max_cluster_size=30,
    alpha=1.0,
    algorithm="brute",
    n_jobs=None,
    cluster_selection_method="eom",  # eom with minsample = 3 is kinda good
    store_centers=None,
)

df_cross, cluster_cross = Cluster.perform_cross_clustering(df_cross=df_cross, params=params)
print(
    Preprocess.sort_df(cluster_cross, conf.CLUSTER_INTEREST)[conf.CLUSTER_INTEREST].to_string()
)

/home/jeans/internship/parking-customer-count/countpassenger/Cluster.py:58: RuntimeWarning: overflow encountered in exp
  sigmoid = 1 / (1 + np.exp(-k * (x - n)))
/home/jeans/internship/parking-customer-count/countpassenger/Cluster.py:60: RuntimeWarning: overflow encountered in exp
  exponential_part = np.exp(x - n) + n - 1


           timestamp_min        timestamp_max  count  cluster_id        xmid_mean        ymid_mean
1    2024-04-29 02:03:45  2024-04-29 02:03:49      3           0  1642.6666666667   500.6666666667
2    2024-04-29 02:29:07  2024-04-29 02:29:47      5           1  2053.1000000000   277.7000000000
6    2024-04-29 02:45:54  2024-04-29 02:50:26      3           5  2110.1666666667   231.5000000000
12   2024-04-29 02:54:58  2024-04-29 02:55:02      2          11  2117.7500000000   135.2500000000
13   2024-04-29 02:55:20  2024-04-29 02:55:21      2          12  1662.0000000000   476.7500000000
35   2024-04-29 03:07:26  2024-04-29 03:08:39      2          34  1869.2500000000   327.7500000000
55   2024-04-29 03:17:13  2024-04-29 03:22:34      3          54  1325.8333333333   644.1666666667
56   2024-04-29 03:29:04  2024-04-29 03:35:09      3          55  1592.8333333333   424.6666666667
0    2024-04-29 03:44:13  2024-04-29 16:43:28     33          -1  1448.1969696970   577.8636363636
83   2024-

/home/jeans/anaconda3/envs/ml_env/lib/python3.12/site-packages/sklearn/cluster/_hdbscan/hdbscan.py:264: UserWarning: The minimum spanning tree contains edge weights with value infinity. Potentially, you are missing too many distances in the initial distance matrix for the given neighborhood size.
  warn(


In [4]:
params = dict(
    min_cluster_size=2,
    min_samples=None,
    cluster_selection_epsilon=0,
    max_cluster_size=30,
    alpha=1.0,
    algorithm="brute",
    cluster_selection_method="eom",
    store_centers=None,
)

df_reverse, cluster_reverse = Cluster.perform_reverse_clustering(df_reverse=df_reverse, params=params)
print(
    Preprocess.sort_df(cluster_reverse, conf.CLUSTER_INTEREST)[
        (cluster_reverse["timestamp_unix_max"] >= start) & (cluster_reverse["timestamp_unix_min"] <= stop)
    ][conf.CLUSTER_INTEREST].to_string()
)

/home/jeans/internship/parking-customer-count/countpassenger/Cluster.py:60: RuntimeWarning: overflow encountered in exp
  exponential_part = np.exp(x - n) + n - 1
/home/jeans/internship/parking-customer-count/countpassenger/Cluster.py:58: RuntimeWarning: overflow encountered in exp
  sigmoid = 1 / (1 + np.exp(-k * (x - n)))


Empty DataFrame
Columns: [timestamp_min, timestamp_max, count, cluster_id, xmid_mean, ymid_mean]
Index: []


/home/jeans/anaconda3/envs/ml_env/lib/python3.12/site-packages/sklearn/cluster/_hdbscan/hdbscan.py:264: UserWarning: The minimum spanning tree contains edge weights with value infinity. Potentially, you are missing too many distances in the initial distance matrix for the given neighborhood size.
  warn(
/tmp/ipykernel_71532/215765330.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  Preprocess.sort_df(cluster_reverse, conf.CLUSTER_INTEREST)[


lifespan of customer's cluster to find the most probable vehicle in the snapshot.
Snapshot cam: takes photo every 1:15 second

nearest x,y ? in time windows?

In [5]:
df_vehicle_snapshot_with_c = Snapshot.match_cross_cluster_snapshot_to_vehicle(
    df_vehicle_snapshot=df_vehicle, cluster_cross=cluster_cross, capture_window=75
)
df_vehicle_snapshot_with_c_and_r = Snapshot.match_reverse_cluster_snapshot_to_vehicle(
    df_vehicle_snapshot=df_vehicle_snapshot_with_c, cluster_reverse=cluster_reverse, capture_window=75
)
df_vehicle_snapshot_with_c_and_r = Preprocess.sort_df(df_vehicle_snapshot_with_c_and_r, ['timestamp_precise','plate_number'])
print(df_vehicle_snapshot_with_c_and_r[conf.VEHICLE_INTEREST_SNAPSHOT_CLUSTERED].to_string())

#Exporting as csv
# df_vehicle_snapshot_with_c_and_r.to_csv(path_or_buf=SAVE_DIR, index=False)
# johnny.to_csv(path_or_buf=SAVE_DIR, index = False,date_format=str)

                          camera       timestamp_precise  cross_count  reverse_count vehicle_type    cluster_cross_list cluster_reverse_list plate_number    xmid    ymid
1577  mbk-14-11-vehicle-snapshot 2024-04-29 02:00:37.635            0              3        other                    []                 [10]          NaN  1769.5   213.0
1578  mbk-14-11-vehicle-snapshot 2024-04-29 02:01:52.615            0             13          bus                    []        [1, 3, 4, 11]          NaN  1760.5   208.0
1579  mbk-14-11-vehicle-snapshot 2024-04-29 02:03:07.593            3              0        truck                   [0]                   []       320790  1763.5   209.5
1561  mbk-14-11-vehicle-snapshot 2024-04-29 02:06:15.104            0              0          NaN                    []                   []          NaN  1615.0   272.5
1562  mbk-14-11-vehicle-snapshot 2024-04-29 02:07:30.083            0              4          NaN                    []                  [0]          